# **PINKSHEPZ'S DATA ANALYTIC**
## Dataset
1. *Candlestick price*: XAUUSD TF 1D - 12 Oct 2020 to 8 May 2023
2. *Economic figure report*: FOREX FACTORY TF 1D - 13 Oct 2020 to 31 Dec 2022

## Analytics
1. Candlestick price distribution
    * Scatter 3D: Time - %Change - Frequency distribution
    * Line 2D: Timezone - Mean %Change
    * Scatter 2D: %Change of n candlestick - %Change of n+1 candlestick
2. Economic figure distribution
3. Candlestick price x economic figure correlation

## **01 IMPORT & DATA PREPARATION**

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
# Import dataset
PATH_XAUUSD = '/workspaces/Enterprise/03_Analytics/input/XAUUSD_080523_121020.csv'
PATH_FOREX = '/workspaces/Enterprise/03_Analytics/input/FOREX_311222_131020.csv'

DF_XAUUSD = pd.read_csv(PATH_XAUUSD).drop(columns=['Vol'])
DF_XAUUSD.iloc[:, 0] = pd.to_datetime(DF_XAUUSD.iloc[:, 0]).dt.date
for col in DF_XAUUSD.columns[1:5]:
    DF_XAUUSD[col] = DF_XAUUSD[col].str.replace(',', '').astype('float64')
DF_XAUUSD['%Change'] = DF_XAUUSD['%Change'].str.replace('%', '').astype('float64')

DF_FOREX = pd.read_csv(PATH_FOREX).iloc[:, 1:]
DF_FOREX.iloc[:, 0] = pd.to_datetime(DF_XAUUSD.iloc[:, 0]).dt.date
DF_FOREX['currency'].astype('category')
DF_FOREX['impact'].astype('category')
DF_FOREX['event'].astype('category')

def handle_number_format(number: str) -> list:
    """Return [number as float, unit i.e. % K M]"""
    # Case 1: na
    if number == '':
        return [None, None]
    number = str(number)
    # Case 2: % value
    if (number[-1] == '%') & (number.lstrip('-')[0].isnumeric() == True):
        return [float(number.rstrip('%')), '%']
    # Case 3: 1K 1M 1B 1T unti
    if (number[-1].isalpha() == True) & (number.lstrip('-')[0].isnumeric() == True):
        return [float(str(number)[:-1]), number[-1]]
    # Case 4: ordinary numbers
    if number.replace(',', '').replace('.', '').lstrip('-').isnumeric() == True: 
        return [float(number.replace(',', '')), None]
    # Case 5: non-numbers
    return [number, None]

for col in DF_FOREX.columns[5:]:
    DF_FOREX[col] = DF_FOREX[col].fillna('').apply(handle_number_format)

## **02 CANDLESTICK EDA**

In [4]:
DF_FOREX

,date,time,currency,impact,event,actual,forecast,previous
0,2023-05-08,8:30,CNY,low,Manufacturing PMI,"[47.0, None]","[48.0, None]","[48.0, None]"
1,2023-05-07,14:00,GBP,low,Nationwide HPI m/m,"[-0.1, %]","[-0.7, %]","[-1.4, %]"
2,2023-05-05,,EUR,low,Spanish Flash CPI y/y,"[5.8, %]","[6.1, %]","[6.8, %]"
3,2023-05-04,21:45,USD,medium,Chicago PMI,"[44.9, None]","[40.1, None]","[37.2, None]"
4,2023-05-03,,CNY,low,Non-Manufacturing PMI,"[41.6, None]","[45.8, None]","[46.7, None]"
...,...,...,...,...,...,...,...,...
10410,NaN,20:30,USD,high,CPI y/y,"[1.4, %]","[1.4, %]","[1.3, %]"
10411,NaN,,USD,medium,CPI m/m,"[0.2, %]","[0.2, %]","[0.4, %]"
10412,NaN,,USD,medium,Core CPI m/m,"[0.2, %]","[0.2, %]","[0.4, %]"
10413,NaN,22:00,GBP,high,BOE Gov Bailey Speaks,"[None, None]","[None, None]","[None, None]"
